# Problem 1

Starting by extracting the emails by using the librarie tarfile.

In [ ]:
import tarfile
import os

for file in os.listdir("/content/"):
  if file.endswith(".bz2"):
    with tarfile.open(file, "r:bz2") as tar:
      tar.extractall(path = "tmp")

extracted_folders = os.listdir("/content/tmp")
print("Extracted folders:", extracted_folders)

Extracted folders: ['easy_ham', 'spam', 'hard_ham']


We extracted the files from their bz2 format and now we are going to export the emails to induvidual text files to be able to analyze thier content and so one can supply an anwser to question A.

In [ ]:
import email
from email import policy
from email.parser import BytesParser
from email.header import decode_header

for folder in extracted_folders:
  path = "/content/tmp/" + folder
  file_list = os.listdir(path)

#Opening one folder at the time and then reading and writing each file to a .txt based on the folder name
#In the end genereates three .txt that can be read and analyzed for question A
  for file_name in file_list:
    with open(path + "/" + file_name, "rb") as file:
      with open(folder + ".txt", "a", encoding="utf-8") as f:
        msg = BytesParser(policy=policy.default).parse(file)

        if msg["subject"] is None:
          continue
        else:
          f.write(f"Subject: {msg['subject']}\n")
        f.write(f"From: {msg['from']}\n")

        if msg['to'] is None:
          continue
        else:
          f.write(f"To: {msg['to']}\n")
        body = msg.get_body(preferencelist=('plain',))

        if body is not None:
          try:
              content = body.get_content()
              charset = body.get_content_charset() or "utf-8"
              #Attempt to decode using the detected charset if not fallback will apply
              content = content.encode("ascii", "ignore").decode(charset, errors="replace")
              f.write(f"Body:\n{content}\n")
          except LookupError:

              f.write("Body: [Could not decode the body due to unknown encoding]\n")

          #Email seperator for easy reading
          f.write("###########################################################\n")

# Problem 1: Question A
Starting by looking at the readme.html provided by spamassassin its stated that the easy_ham emails dont frequently contain spammish signatures which can be lots of capital letters, html and keyword like "adult-only" etc. Hard_ham on the other hand have serveral aspects close to spam. So some of the signatures of spam can be found in the emails, which makes them harder to detect as non spam. And the spam file includes spam.



**Sample mail from the txt files can be seen below:**

---



Subject: Rats' intestines and pigs' teeth
From: boingboing <rssfeeds@example.com>
To: yyyy@example.com
Body:
URL: http://boingboing.net/#85497383
Date: Not supplied

This is the headline of the month, possibly the year: "Doctors Grow Pig Teeth
in Rat Intestines." Do we even need to read the story to understand it? It's
like a freaking _haiku_ of near-singularity, future-shocky wonderment!

    U.S. doctors said on Thursday they have managed to grow living pig teeth in
    rats, a feat of biotechnology that experts said could spark a dental
    revolution.

    Researchers at Boston's Forsyth Institute said their successful experiment
    suggests the existence of dental stem cells, which could one day allow a
    person to replace a lost tooth with an identical one grown from his or her
    own cells.

    "The ability to identify, isolate and propagate dental stem cells to use in
    biological replacement tooth therapy has the potential to revolutionize
    dentistry," said Dominick DePaola, president and CEO of the institute that
    focuses on oral and facial science.  

Link[1] Discuss[2] (_Thanks, Dave[3]!_)

[1] http://story.news.yahoo.com/news?tmpl=story&ncid=585&e=1&cid=585&u=/nm/20020926/sc_nm/health_teeth_dc
[2] http://www.quicktopic.com/boing/H/88cUs6CumjiX
[3] http://www.remtullaeurorscg.com

# Problem 1: Question B
Start with creating python lists that include spam and easy-ham and the other will also include spam and hard-ham. The following code have resamblens to the code seen in "Problem 1" but for more structured code this was the aproch used. After the mails are put into the list the data spliting is perfomed. As mentioned in the lecture the trainset consist of 75% of the mails and only 25% for the test.

In [ ]:
from sklearn import model_selection as ms
from sklearn.model_selection import train_test_split

#Create 2 list which is gonna hold mail from easy and hard, both the list will also contain spam for the data splitting.
lst_ez_ham = []
lst_hd_ham = []
lst_spam = []

for folder in extracted_folders:
  for file in os.listdir("/content/tmp/" + folder + "/"):
    with open("/content/tmp/" + folder + "/" + file, "rb") as f:
      data = f.readlines()
      data = [line.decode('utf-8', errors='ignore') for line in data]  # Decode binary data to text
      content = ''.join(data)  # Join lines into a single string


      if folder == "easy_ham":
        lst_ez_ham.append(content)
      elif folder == "hard_ham":
        lst_hd_ham.append(content)
      else:
        lst_spam.append(content)


#Rule of thumb: if we do a simple train-test split, use 75% of data for the training set and 25% for the test set

#Easy Ham Split datasplit
X_ez = lst_ez_ham + lst_spam
y_ez = [0] * len(lst_ez_ham) + [1] * len(lst_spam) #Labels

X_ez_train, X_ez_test, y_ez_train, y_ez_test = train_test_split(X_ez, y_ez, test_size=0.25, random_state=64, stratify=y_ez)

#Hard Ham Split datasplit
X_hd = lst_hd_ham + lst_spam
y_hd = [0] * len(lst_hd_ham) + [1] * len(lst_spam) #Labels

X_hd_train, X_hd_test, y_hd_train, y_hd_test = train_test_split(X_hd, y_hd, test_size=0.25, random_state=64, stratify=y_hd)

Labels are important for the model to differentiate between ham and spam based on the content in the training dataset.

# Problem 2: Preprocessing

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import os
import tarfile

In [ ]:
os.makedirs("dataset/spam", exist_ok=True)
os.makedirs("dataset/ham", exist_ok=True)
with tarfile.open("/content/dataset/easy_ham.tar.gz") as tar_ref:
    tar_ref.extractall("/content/dataset/easy_ham")
    tar_ref.close()

with tarfile.open("/content/dataset/spam.tar.gz") as tar_ref:
    tar_ref.extractall("/content/dataset/spam")
    tar_ref.close()

Here we do some preprocessing. We create labels - 1 for ham and 0 for spam. We also create and training and test split with the the help of train_test_split(). We set the random state value to 64 so that when we run the program again we get the same split so that we can reproduce results.

In [ ]:
spam_list = []
ham_list = []

for file in os.listdir("/content/dataset/spam/spam"):
    with open(os.path.join("/content/dataset/spam/spam", file), 'r', errors="ignore") as file:
        spam_list.append(file.read())

for file in os.listdir("/content/dataset/easy_ham/easy_ham"):
   with open(os.path.join("/content/dataset/easy_ham/easy_ham", file), 'r', errors="ignore") as file:
       ham_list.append(file.read())

ham_labels = [1] * len(ham_list)
spam_labels = [0] * len(spam_list)

all_mail = spam_list + ham_list
all_labels = ham_labels + spam_labels

train_files, test_files, train_labels, test_labels = train_test_split(
    all_mail, all_labels, test_size=0.25, random_state=64, stratify=all_labels
)


# Problem 3: Easy Ham
Here we perform the actual training of the model and then predict. We first use fit_transform(train_files) to train the model on the training data. We then transform the model, using transform(test_files) (not fit_transform as that would fit the model to the test data thus making the test useless) and finally make a prediction and observe the metrics.

We also have a .75/.25 training/test split as suggested in the lecture.

In [ ]:
vectorizer = CountVectorizer()
x_train = vectorizer.fit_transform(train_files)
x_test = vectorizer.transform(test_files)
print(vectorizer.get_feature_names_out())
print(x_train.toarray())


clf = MultinomialNB()
clf.fit(x_train, train_labels)
test_predictions = clf.predict(x_test)

accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy for Multinomial: {accuracy * 100:.2f}%")
############################################################################################

clfB = BernoulliNB()
clfB.fit(x_train, train_labels)
testB_predictions = clfB.predict(x_test)
accuracyB = accuracy_score(test_labels, testB_predictions)
print(f"Accuracy for Bernoulli: {accuracyB * 100:.2f}%")

NameError: name 'train_files' is not defined

Here we see that the accuracy for both models. We also got different accuracies for different values for random_states, which makes sense (to some degree).

# Problem 4: Hard Ham

In [ ]:
vectorizer = CountVectorizer()
x_hd_train = vectorizer.fit_transform(X_hd_train)
x_hd_test = vectorizer.transform(X_hd_test)
print(vectorizer.get_feature_names_out())
print(x_hd_train.toarray())

clf = MultinomialNB()
clf.fit(x_hd_train, y_hd_train)
testA_predictions = clf.predict(x_hd_test)
accuracy = accuracy_score(y_hd_test, testA_predictions)
print(f"Accuracy for Multinomial: {accuracy * 100:.2f}%")

clfB = BernoulliNB()
clfB.fit(x_hd_train, y_hd_train)
testB_predictions = clfB.predict(x_hd_test)
accuracyB = accuracy_score(y_hd_test, testB_predictions)
print(f"Accuracy for Bernoulli: {accuracyB * 100:.2f}%")

['00' '000' '0000' ... '쵥λ' '𬣡b' '𬣤jӡa']
[[ 3  0  2 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 [ 1  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 3  0  0 ...  0  0  0]
 [12  0  8 ...  0  0  0]]
Accuracy for Multinomial: 92.55%
Accuracy for Bernoulli: 84.04%


# Problem 3: Easy Ham

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
x_ez_train = vectorizer.fit_transform(X_ez_train)
x_ez_test = vectorizer.transform(X_ez_test)
print(vectorizer.get_feature_names_out())
print(x_ez_train.toarray())
print("")

clf = MultinomialNB()
clf.fit(x_ez_train, y_ez_train)
testA_predictions = clf.predict(x_hd_test)

accuracy = accuracy_score(y_ez_test, testA_predictions)
precision = precision_score(y_ez_test, testA_predictions)
recall = recall_score(y_ez_test, testA_predictions)
print("The Multinomial Naive Bayesian Classifier")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")

conf_matrix = confusion_matrix(y_ez_test, testA_predictions)
print("Confusion Matrix:")
print(conf_matrix)
print("")
#############################################################

clfB = BernoulliNB()
clfB.fit(x_ez_train, y_ez_train)
testB_predictions = clfB.predict(x_hd_test)

accuracy = accuracy_score(y_ez_test, testB_predictions)
precision = precision_score(y_ez_test, testB_predictions)
recall = recall_score(y_ez_test, testB_predictions)
print("The Bernoulli Naive Bayesian Classifier")
print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision * 100:.2f}%")
print(f"Recall: {recall * 100:.2f}%")

conf_matrix = confusion_matrix(y_ez_test, testB_predictions)
print("Confusion Matrix:")
print(conf_matrix)

['00' '000' '0000' ... '쵥λ' '𬣡b' '𬣤jӡa']
[[0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [3 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [3 0 1 ... 0 0 0]]

The Multinomial Naive Bayesian Classifier
Accuracy: 96.85%
Precision: 98.10%
Recall: 82.40%
Confusion Matrix:
[[636   2]
 [ 22 103]]

The Bernoulli Naive Bayesian Classifier
Accuracy: 90.69%
Precision: 100.00%
Recall: 43.20%
Confusion Matrix:
[[638   0]
 [ 71  54]]
